# AI Trading Strategy

## Objective
The objective of this project is to develop an AI-driven intraday trading strategy using reinforcement learning and deep learning techniques. The focus is on leveraging the IVV ETF data to create models that predict short-term market movements and optimize trading decisions.

## Team Composition
- Emil Alizada
- Ritwick Haldar
- Muhammad Zubair Ahmed Khan


In [ ]:
%run main.py

## Step 1: Setting Environment Variables
Set environment variables to control TensorFlow and matplotlib behaviors, ensuring that TensorFlow runs on the CPU and that matplotlib runs in an offscreen mode.


In [ ]:
# Set environment variables
np.seterr(divide='ignore', invalid='ignore')
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress all logs except errors
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # Force TensorFlow to use CPU
os.environ['QT_QPA_PLATFORM'] = 'offscreen'  # Suppress Qt backend issues


## Step 2: Load Data
Load the training and validation datasets for the IVV ETF. We use a subset of the data for testing purposes.


In [ ]:
# Load data
train_data = pd.read_csv('IVV_1m_training.csv') # Using a subset for testing
val_data = pd.read_csv('IVV_1m_validation.csv')  # Using a subset for testing


## Step 3: Data Preprocessing
Preprocess the data by adding technical indicators such as Simple Moving Average (SMA), Relative Strength Index (RSI), Moving Average Convergence Divergence (MACD), and Bollinger Bands.


In [ ]:
def preprocess_data(data):
    data['DateTime'] = pd.to_datetime(data['DateTime'])
    data.set_index('DateTime', inplace=True)
    data['SMA'] = data['Close'].rolling(window=20).mean()
    data['RSI'] = compute_RSI(data['Close'], 14)
    data['MACD'] = compute_MACD(data['Close'])
    data['BB_upper'], data['BB_middle'], data['BB_lower'] = compute_Bollinger_Bands(data['Close'])
    data.dropna(inplace=True)
    return data

def compute_RSI(data, window):
    diff = data.diff(1)
    gain = diff.clip(lower=0)
    loss = -1 * diff.clip(upper=0)
    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def compute_MACD(data, slow=26, fast=12, signal=9):
    ema_fast = data.ewm(span=fast, min_periods=fast).mean()
    ema_slow = data.ewm(span=slow, min_periods=slow).mean()
    macd = ema_fast - ema_slow
    signal_line = macd.ewm(span=signal, min_periods=signal).mean()
    return macd - signal_line

def compute_Bollinger_Bands(data, window=20, num_std=2):
    rolling_mean = data.rolling(window=window).mean()
    rolling_std = data.rolling(window=window).std()
    upper_band = rolling_mean + (rolling_std * num_std)
    lower_band = rolling_mean - (rolling_std * num_std)
    return upper_band, rolling_mean, lower_band

train_data = preprocess_data(train_data)
val_data = preprocess_data(val_data)


## Step 4: Data Normalization
Normalize the data using `StandardScaler` to ensure that all features have a mean of 0 and a standard deviation of 1, which is essential for training the LSTM model effectively.


In [ ]:
# Normalize data
scaler = StandardScaler()
train_data_scaled = scaler.fit_transform(train_data)
val_data_scaled = scaler.transform(val_data)
joblib.dump(scaler, "scaler.joblib")


## Step 5: Breakout Strategy
Implement a breakout strategy to label the data with buy and sell signals. This strategy identifies potential breakout points based on the highest and lowest prices over a specified look-back period.


In [ ]:
def breakout_strategy(data, lookback=20):
    data['High_max'] = data['High'].rolling(window=lookback).max()
    data['Low_min'] = data['Low'].rolling(window=lookback).min()
    data['Buy'] = np.where(data['Close'] > data['High_max'].shift(1), 1, 0)
    data['Sell'] = np.where(data['Close'] < data['Low_min'].shift(1), 1, 0)
    return data

train_data = breakout_strategy(train_data)
val_data = breakout_strategy(val_data)


## Step 6: Prepare Data for LSTM
Create a dataset for the LSTM model with the specified look-back period. This step involves reshaping the data into sequences that the LSTM model can process.


In [ ]:
def create_dataset(data, look_back=1):
    X, Y = [], []
    for i in range(len(data) - look_back - 1):
        a = data[i:(i + look_back), :-1]
        X.append(a)
        Y.append(data[i + look_back, -1])
    return np.array(X), np.array(Y)

look_back = 60  # 60 minutes look back
X_train, y_train = create_dataset(train_data_scaled, look_back)
X_val, y_val = create_dataset(val_data_scaled, look_back)


## Step 7: Build and Train LSTM Model
Build the LSTM model using Keras, compile it with the Adam optimizer and mean squared error loss function, and then train the model on the prepared training data.


In [ ]:
# Build LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(look_back, X_train.shape[2])))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(25))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, batch_size=64, epochs=50, validation_data=(X_val, y_val))

# Save the model using Keras' save method
model.save('lstm_model.h5')
print("LSTM model saved successfully.")

## Step 8: Define Trading Environment
Create a custom trading environment using the OpenAI Gym framework. This environment simulates trading activities and evaluates the performance of the reinforcement learning agent.


In [ ]:
class TradingEnv(gym.Env):
    def __init__(self, data, initial_balance=10000, stop_loss=0.005, take_profit=0.01, min_trades=2, max_trades=8, trading_cost=0.0001):
        super(TradingEnv, self).__init__()
        self.data = data
        self.current_step = 0
        self.initial_balance = initial_balance
        self.balance = initial_balance
        self.stop_loss = stop_loss
        self.take_profit = take_profit
        self.position = 0  # 0: no position, 1: long, -1: short
        self.entry_price_long = 0
        self.entry_price_short = 0
        self.trades_per_day = 0
        self.min_trades = min_trades
        self.max_trades = max_trades
        self.current_day = self.current_step
        self.daily_trades = []
        self.daily_profits = []
        self.trading_cost = trading_cost  # Trading cost percentage

        self.action_space = spaces.Discrete(3)  # buy, sell, hold
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(look_back, data.shape[1]), dtype=np.float32)

    def reset(self):
        self.current_step = 0
        self.balance = self.initial_balance
        self.position = 0
        self.entry_price_long = 0
        self.entry_price_short = 0
        self.trades_per_day = 0
        self.daily_trades = []
        self.daily_profits = []
        return self.data[self.current_step:self.current_step + look_back]

    def step(self, action):
        reward = 0
        done = self.current_step >= len(self.data) - look_back - 1

        # Take action
        current_price = self.data[self.current_step][-1]
        if action == 1:  # buy
            if self.position == 0 and self.trades_per_day < self.max_trades:
                self.position = 1
                self.entry_price_long = current_price
                self.trades_per_day += 1
            elif self.position == -1:  # if already short, open long too
                self.position = 2  # indicating both long and short positions
                self.entry_price_long = current_price
        elif action == 2:  # sell
            if self.position == 0 and self.trades_per_day < self.max_trades:
                self.position = -1
                self.entry_price_short = current_price
                self.trades_per_day += 1
            elif self.position == 1:  # if already long, open short too
                self.position = 2  # indicating both long and short positions
                self.entry_price_short = current_price
        elif action == 0:  # hold
            pass

        # Check stop-loss and take-profit conditions
        if self.position == 1:
            if (current_price <= self.entry_price_long * (1 - self.stop_loss)) or (current_price >= self.entry_price_long * (1 + self.take_profit)):
                trade_value = min(self.balance, self.balance * (1 - self.trading_cost))  # Adjusting for trading cost
                reward = (current_price - self.entry_price_long) * (trade_value / self.entry_price_long)
                self.balance += reward
                self.position = 0
        elif self.position == -1:
            if (current_price >= self.entry_price_short * (1 + self.stop_loss)) or (current_price <= self.entry_price_short * (1 - self.take_profit)):
                trade_value = min(self.balance, self.balance * (1 - self.trading_cost))  # Adjusting for trading cost
                reward = (self.entry_price_short - current_price) * (trade_value / self.entry_price_short)
                self.balance += reward
                self.position = 0
        elif self.position == 2:
            # Check long position
            if (current_price <= self.entry_price_long * (1 - self.stop_loss)) or (current_price >= self.entry_price_long * (1 + self.take_profit)):
                trade_value_long = min(self.balance, self.balance * (1 - self.trading_cost))  # Adjusting for trading cost
                reward_long = (current_price - self.entry_price_long) * (trade_value_long / self.entry_price_long)
                if reward_long < 0:  # if long is in loss, close it
                    self.position = -1
                else:  # if long is in profit, open another long
                    self.balance += reward_long
                    reward += reward_long
                    self.position = 1  # keep the short position open
            # Check short position
            if (current_price >= self.entry_price_short * (1 + self.stop_loss)) or (current_price <= self.entry_price_short * (1 - self.take_profit)):
                trade_value_short = min(self.balance, self.balance * (1 - self.trading_cost))  # Adjusting for trading cost
                reward_short = (self.entry_price_short - current_price) * (trade_value_short / self.entry_price_short)
                if reward_short < 0:  # if short is in loss, close it
                    self.position = 1
                else:  # if short is in profit, open another short
                    self.balance += reward_short
                    reward += reward_short
                    self.position = -1  # keep the long position open

        self.current_step += 1

                        # Check if it's a new day
        if self.current_step % 1440 == 0:  # Assuming one day is 1440 minutes
            if self.trades_per_day < self.min_trades:
                reward -= 100  # Penalty for not meeting the minimum trades requirement
            if self.position != 0:
                # Close position at market price at the end of the day
                if self.position == 1:
                    trade_value = min(self.balance, self.balance * (1 - self.trading_cost))  # Adjusting for trading cost
                    reward += (current_price - self.entry_price_long) * (trade_value / self.entry_price_long)
                elif self.position == -1:
                    trade_value = min(self.balance, self.balance * (1 - self.trading_cost))  # Adjusting for trading cost
                    reward += (self.entry_price_short - current_price) * (trade_value / self.entry_price_short)
                reward -= abs(reward * self.trading_cost)  # Deduct trading cost
                self.balance += reward
                self.position = 0
            self.daily_trades.append(self.trades_per_day)
            self.daily_profits.append(self.balance - self.initial_balance)
            self.trades_per_day = 0

        state = self.data[self.current_step:self.current_step + look_back]
        return state, reward, done, {}


## Step 9: Load Trained Model and Test Environment
Load the trained model and evaluate it in the custom trading environment. This involves running the reinforcement learning agent and recording the actions, rewards, and overall performance.


In [ ]:
# Create the trading environment
train_env = TradingEnv(train_data_scaled)
val_env = TradingEnv(val_data_scaled)

# Train the PPO model
drl_model = PPO('MlpPolicy', train_env, verbose=1)
drl_model.learn(total_timesteps=10000)  # Adjust timesteps as needed

# Save the trained model
drl_model.save("drl_trading_model")
print("Model trained and saved successfully.")
# Load the trained model

%matplotlib inline


drl_model = PPO.load("drl_trading_model")
val_env = TradingEnv(val_data_scaled)

state = val_env.reset()
done = False

cumulative_return = []
actions = []
profits = []
initial_balance = 10000  # Starting with $10,000
successful_trades = 0
total_trades = 0

current_balance = initial_balance

# Define variables to track net profit
net_profit = 0
total_trade_amount = 0

while not done:
    action, _states = drl_model.predict(state)
    state, reward, done, info = val_env.step(action)
    current_balance += reward
    profits.append(reward)
    cumulative_return.append(current_balance)
    actions.append(action)

    # Update successful trades count
    if reward > 0:
        successful_trades += 1
    total_trades += 1

    # Update net profit and total trade amount
    if reward != 0:
        net_profit += reward
        total_trade_amount += np.abs(reward)

    # Print trade details
    if reward != 0:
        current_date_time = val_data.index[val_env.current_step]
        print(f"Trade Details:")
        print(f"Date and Time: {current_date_time}")
        print(f"Action: {'Buy' if action == 1 else 'Sell'}")
        print(f"Reward: {reward}")

# Calculate cumulative return in percentage
cumulative_return = np.array(cumulative_return)
cumulative_return = (cumulative_return - initial_balance) / initial_balance * 100  # Percentage return

# Calculate success accuracy
success_accuracy = (successful_trades / total_trades) * 100

# Store results in a DataFrame
results = pd.DataFrame({
    'cumulative_return': cumulative_return,
    'actions': actions,
    'profits': profits
})

import matplotlib.pyplot as plt

# Plotting buy and sell signals on training data
plt.figure(figsize=(20, 10))
plt.plot(train_data.index, train_data['Close'], label='Close')
plt.scatter(train_data.index[train_data['Buy'] == 1], train_data['Close'][train_data['Buy'] == 1], marker='^', color='g', label='Buy', alpha=1)
plt.scatter(train_data.index[train_data['Sell'] == 1], train_data['Close'][train_data['Sell'] == 1], marker='v', color='r', label='Sell', alpha=1)
plt.title('Training data')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()  # Show the plot

# Plotting buy and sell signals on validation data
plt.figure(figsize=(20, 10))
plt.plot(val_data.index, val_data['Close'], label='Close')
plt.scatter(val_data.index[val_data['Buy'] == 1], val_data['Close'][val_data['Buy'] == 1], marker='^', color='g', label='Buy', alpha=1)
plt.scatter(val_data.index[val_data['Sell'] == 1], val_data['Close'][val_data['Sell'] == 1], marker='v', color='r', label='Sell', alpha=1)
plt.title('Validation data')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()  # Show the plot

# Plot actions taken by the agent
plt.figure(figsize=(12, 6))
plt.plot(results.index, results['actions'], label='Actions')
plt.xlabel('Time')
plt.ylabel('Action')
plt.title('Actions Taken by the Trading Agent')
plt.legend()
plt.show()  # Show the plot

# Plot profits per trade
plt.figure(figsize=(12, 6))
plt.plot(results.index, results['profits'], label='Profit per Trade')
plt.xlabel('Time')
plt.ylabel('Profit')
plt.title('Profit per Trade Over Time')
plt.legend()
plt.show()  # Show the plot

print("Plots displayed successfully.")

print(f"Success Accuracy: {success_accuracy:.2f}%")

# Example of detailed output
net_worth = initial_balance + net_profit
balance = val_env.balance
trades_per_day = np.mean(val_env.daily_trades)

print(f"Net Profit: {net_profit}")
print(f"Net Worth: {net_worth}")
print(f"Balance: {balance}")
print(f"Trades: {total_trades}")
print(f"Trades per day: {trades_per_day}")

np.seterr(divide='warn',invalid='warn')

## Step 10: Calculate Metrics
Calculate important trading performance metrics such as cumulative returns, compounded annual return, annual volatility, Value at Risk (VaR), Conditional Value at Risk (CVaR), Sharpe Ratio, and Sortino Ratio.


In [ ]:
# Calculate metrics
# Calculate cumulative returns (%)
final_balance = net_worth  # Assuming net_worth represents the final balance
cumulative_returns = ((final_balance - initial_balance) / initial_balance) * 1

# Compounded annual return (%)
start_date = pd.to_datetime(results.index[0])
end_date = pd.to_datetime(results.index[-1])
num_days = (end_date - start_date).days
num_years = num_days / 365.25 if num_days > 365.25 else 1  # Prevent division by zero
compounded_annual_return = ((net_worth / initial_balance) ** (1 / num_years) - 1) * 100

# Annual volatility
annual_volatility = np.std(results['cumulative_return'].pct_change().dropna()) * np.sqrt(252) * 100  # Assuming 252 trading days in a year

# VaR Hist (Value at Risk)
var_hist = np.percentile(results['cumulative_return'], 5)  # 5% VaR

# CVaR Hist (Conditional Value at Risk)
cvar_hist = results['cumulative_return'][results['cumulative_return'] <= var_hist].mean()

# Sharpe Ratio
daily_returns = results['cumulative_return'].pct_change().dropna()
sharpe_ratio = (daily_returns.mean() / daily_returns.std()) * np.sqrt(252)  # Assuming 252 trading days in a year

# Sortino Ratio
downside_returns = daily_returns[daily_returns < 0]
sortino_ratio = (daily_returns.mean() / downside_returns.std()) * np.sqrt(252)

# Print or use the calculated metrics as needed
print("Cumulative Returns (%):", cumulative_returns)
print("Compounded Annual Return (%):", compounded_annual_return)
print("Annual Volatility:", annual_volatility)
print("VaR Hist:", var_hist)
print("CVaR Hist:", cvar_hist)
print("Sharpe Ratio:", sharpe_ratio)
print("Sortino Ratio:", sortino_ratio)
